# Análisis Completo de Circuitos con 2 Amplificadores Operacionales

Este notebook realiza un análisis completo de circuitos con dos amplificadores operacionales, incluyendo:
- Equivalentes Thévenin
- Función de transferencia
- Análisis de polos y ceros
- Estabilidad
- Respuestas temporales y frecuenciales

In [2]:
# Importar las bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from sympy import symbols, solve, expand, simplify, limit
from sympy.abc import s
import control
from control import TransferFunction
%matplotlib inline
plt.style.use('seaborn-darkgrid')

OSError: 'seaborn-darkgrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## 1. Definición del Circuito

Definiremos un circuito con dos amplificadores operacionales y sus componentes. Los usuarios pueden modificar estos valores según su circuito específico.

In [ ]:
# Definición de componentes
R1, R2, R3, R4, R5, R6 = symbols('R1 R2 R3 R4 R5 R6')
C1, C2 = symbols('C1 C2')

# Valores numéricos (ejemplo)
valores = {
    R1: 10e3,  # 10 kΩ
    R2: 20e3,  # 20 kΩ
    R3: 15e3,  # 15 kΩ
    R4: 30e3,  # 30 kΩ
    R5: 25e3,  # 25 kΩ
    R6: 40e3,  # 40 kΩ
    C1: 100e-9,  # 100 nF
    C2: 47e-9   # 47 nF
}

## 2. Equivalentes Thévenin

### 2.1 Equivalente Thévenin desde la entrada

In [ ]:
# Cálculo del equivalente Thévenin desde la entrada
def calc_thevenin_entrada(R1, R2, C1, s):
    # Impedancia del capacitor
    Z_C1 = 1/(s*C1)
    
    # Impedancia Thévenin
    Z_th = (R1 * (R2 + Z_C1))/(R1 + R2 + Z_C1)
    
    # Tensión Thévenin (asumiendo fuente de entrada Vin = 1V)
    V_th = R2/(R1 + R2 + Z_C1)
    
    return V_th, Z_th

# Calcular equivalente Thévenin para diferentes frecuencias
freq = np.logspace(0, 5, 1000)
w = 2*np.pi*freq
Z_th_mag = []
Z_th_phase = []

for f in freq:
    V_th, Z_th = calc_thevenin_entrada(
        valores[R1], 
        valores[R2], 
        valores[C1], 
        2*np.pi*f*1j
    )
    Z_th_mag.append(abs(Z_th))
    Z_th_phase.append(np.angle(Z_th, deg=True))

# Graficar impedancia Thévenin
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.semilogx(freq, Z_th_mag)
ax1.set_title('Magnitud de la Impedancia Thévenin de Entrada')
ax1.set_xlabel('Frecuencia (Hz)')
ax1.set_ylabel('|Z_th| (Ω)')
ax1.grid(True)

ax2.semilogx(freq, Z_th_phase)
ax2.set_title('Fase de la Impedancia Thévenin de Entrada')
ax2.set_xlabel('Frecuencia (Hz)')
ax2.set_ylabel('Fase (grados)')
ax2.grid(True)

plt.tight_layout()
plt.show()

### 2.2 Equivalente Thévenin desde la salida

In [ ]:
# Cálculo del equivalente Thévenin desde la salida
def calc_thevenin_salida(R5, R6, C2, s):
    # Impedancia del capacitor
    Z_C2 = 1/(s*C2)
    
    # Impedancia Thévenin
    Z_th = (R5 * (R6 + Z_C2))/(R5 + R6 + Z_C2)
    
    # Tensión Thévenin (asumiendo tensión en el segundo op-amp = 1V)
    V_th = R6/(R5 + R6 + Z_C2)
    
    return V_th, Z_th

# Calcular equivalente Thévenin de salida para diferentes frecuencias
Z_th_salida_mag = []
Z_th_salida_phase = []

for f in freq:
    V_th, Z_th = calc_thevenin_salida(
        valores[R5], 
        valores[R6], 
        valores[C2], 
        2*np.pi*f*1j
    )
    Z_th_salida_mag.append(abs(Z_th))
    Z_th_salida_phase.append(np.angle(Z_th, deg=True))

# Graficar impedancia Thévenin de salida
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.semilogx(freq, Z_th_salida_mag)
ax1.set_title('Magnitud de la Impedancia Thévenin de Salida')
ax1.set_xlabel('Frecuencia (Hz)')
ax1.set_ylabel('|Z_th| (Ω)')
ax1.grid(True)

ax2.semilogx(freq, Z_th_salida_phase)
ax2.set_title('Fase de la Impedancia Thévenin de Salida')
ax2.set_xlabel('Frecuencia (Hz)')
ax2.set_ylabel('Fase (grados)')
ax2.grid(True)

plt.tight_layout()
plt.show()

## 3. Función de Transferencia

Calcularemos la función de transferencia del circuito completo.

In [ ]:
# Función de transferencia simbólica
def calc_transfer_function():
    # Definir variable s para Laplace
    s = symbols('s')
    
    # Primera etapa (primer op-amp)
    Z_C1 = 1/(s*C1)
    H1 = -R2/R1 * (1/(1 + s*R2*C1))
    
    # Segunda etapa (segundo op-amp)
    Z_C2 = 1/(s*C2)
    H2 = -R6/R5 * (1/(1 + s*R6*C2))
    
    # Función de transferencia total
    H = H1 * H2
    
    return simplify(H)

# Calcular función de transferencia
H = calc_transfer_function()
print("Función de transferencia:")
print(H)

# Crear función de transferencia numérica para control.TransferFunction
def get_numeric_tf():
    H = calc_transfer_function()
    H_num = H.subs(valores)
    
    # Convertir a coeficientes numéricos
    num, den = signal.expand(H_num).as_numer_denom()
    num_coeff = [complex(x).real for x in Poly(num, s).all_coeffs()]
    den_coeff = [complex(x).real for x in Poly(den, s).all_coeffs()]
    
    return TransferFunction(num_coeff, den_coeff)

# Obtener función de transferencia numérica
sys = get_numeric_tf()

# Graficar respuesta en frecuencia
w = np.logspace(-1, 5, 1000)
mag, phase, w = control.bode(sys, w, plot=True, dB=True)
plt.show()

## 4. Polos y Ceros

Analizaremos los polos y ceros de la función de transferencia.

In [ ]:
# Calcular polos y ceros
zeros = control.zero(sys)
poles = control.pole(sys)

print("Ceros del sistema:")
for i, zero in enumerate(zeros, 1):
    print(f"z{i} = {zero:.2f}")

print("\nPolos del sistema:")
for i, pole in enumerate(poles, 1):
    print(f"p{i} = {pole:.2f}")

# Graficar diagrama de polos y ceros
plt.figure(figsize=(8, 8))
control.pzmap(sys, plot=True)
plt.title('Diagrama de Polos y Ceros')
plt.grid(True)
plt.show()

## 5. Análisis de Estabilidad

Analizaremos la estabilidad del sistema utilizando los polos y el criterio de Routh-Hurwitz.

In [ ]:
# Análisis de estabilidad
def analizar_estabilidad(poles):
    # Verificar si hay polos en el semiplano derecho
    stable = all(pole.real < 0 for pole in poles)
    
    # Calcular márgenes de estabilidad
    gm, pm, wg, wp = control.margin(sys)
    
    print("Análisis de estabilidad:")
    print(f"Sistema {'estable' if stable else 'inestable'}")
    print(f"\nMargen de ganancia: {gm:.2f} dB")
    print(f"Margen de fase: {pm:.2f} grados")
    print(f"Frecuencia de cruce de ganancia: {wg:.2f} rad/s")
    print(f"Frecuencia de cruce de fase: {wp:.2f} rad/s")
    
    return stable, gm, pm, wg, wp

# Realizar análisis de estabilidad
stable, gm, pm, wg, wp = analizar_estabilidad(poles)

# Graficar márgenes de estabilidad
plt.figure(figsize=(12, 8))
control.margin(sys)
plt.show()

## 6. Respuestas Temporales

Analizaremos las respuestas naturales, al escalón y al impulso del sistema.

In [ ]:
# Generar vector de tiempo
t = np.linspace(0, 0.01, 1000)

# Respuesta natural (condiciones iniciales no nulas)
t_natural, y_natural = control.initial_response(sys, t)

# Respuesta al escalón
t_step, y_step = control.step_response(sys, t)

# Respuesta al impulso
t_impulse, y_impulse = control.impulse_response(sys, t)

# Graficar todas las respuestas
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))

ax1.plot(t_natural, y_natural)
ax1.set_title('Respuesta Natural')
ax1.set_xlabel('Tiempo (s)')
ax1.set_ylabel('Amplitud')
ax1.grid(True)

ax2.plot(t_step, y_step)
ax2.set_title('Respuesta al Escalón')
ax2.set_xlabel('Tiempo (s)')
ax2.set_ylabel('Amplitud')
ax2.grid(True)

ax3.plot(t_impulse, y_impulse)
ax3.set_title('Respuesta al Impulso')
ax3.set_xlabel('Tiempo (s)')
ax3.set_ylabel('Amplitud')
ax3.grid(True)

plt.tight_layout()
plt.show()

# Calcular características de la respuesta al escalón
info = control.step_info(sys)
print("\nCaracterísticas de la respuesta al escalón:")
for key, value in info.items():
    print(f"{key}: {value:.3f}")

## 7. Análisis de Respuestas Forzadas

### 7.1 Ganancia DC
### 7.2 Ganancia AC (Análisis Fasorial)
### 7.3 Ganancia Exponencial

In [ ]:
# Ganancia DC
dc_gain = control.dcgain(sys)
print(f"Ganancia DC: {dc_gain:.2f} dB")

# Análisis AC (respuesta en frecuencia)
w = np.logspace(-1, 5, 1000)
mag, phase, _ = control.bode(sys, w, plot=False)

# Convertir magnitud a dB
mag_db = 20 * np.log10(mag)

# Ganancia AC a diferentes frecuencias
print("\nGanancia AC a frecuencias específicas:")
freqs = [1, 10, 100, 1000, 10000]  # Hz
for f in freqs:
    idx = np.abs(w - 2*np.pi*f).argmin()
    print(f"A {f} Hz: {mag_db[idx]:.2f} dB, Fase: {phase[idx]:.2f} grados")

# Respuesta a entrada exponencial
# Simulamos la respuesta a e^(at) para diferentes valores de a
def exp_response(sys, a, t):
    # Crear sistema modificado para entrada exponencial
    num, den = control.tfdata(sys)
    sys_mod = control.TransferFunction(num[0], np.polyder(den[0]))
    
    # Calcular respuesta
    _, y = control.forced_response(sys_mod, t, U=np.exp(a*t))
    return y

# Calcular respuestas para diferentes exponentes
t = np.linspace(0, 0.01, 1000)
a_values = [-100, -10, 10, 100]
plt.figure(figsize=(10, 6))

for a in a_values:
    y = exp_response(sys, a, t)
    plt.plot(t, y, label=f'a = {a}')

plt.title('Respuesta a Entradas Exponenciales')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.legend()
plt.grid(True)
plt.show()

## 8. Respuesta en Frecuencia y Análisis del Tipo de Filtro

Analizaremos la respuesta en frecuencia detallada y determinaremos el tipo de filtro.

In [ ]:
# Análisis detallado de la respuesta en frecuencia
w = np.logspace(-1, 5, 1000)
w, mag, phase = control.bode(sys, w, plot=False)

# Convertir frecuencia angular a Hz y magnitud a dB
freq = w/(2*np.pi)
mag_db = 20 * np.log10(mag)

# Encontrar frecuencia de corte (-3dB)
cutoff_idx = np.abs(mag_db - (max(mag_db) - 3)).argmin()
cutoff_freq = freq[cutoff_idx]

# Determinar tipo de filtro
def determine_filter_type(mag_db, freq):
    # Analizar pendiente en altas frecuencias
    high_freq_slope = np.mean(np.diff(mag_db[-100:]))/np.mean(np.diff(np.log10(freq[-100:])))
    
    # Analizar comportamiento en bajas y altas frecuencias
    low_freq_gain = mag_db[0]
    high_freq_gain = mag_db[-1]
    
    if abs(high_freq_slope) < 10:
        return "Pasa todo"
    elif high_freq_gain > low_freq_gain:
        return "Pasa altas"
    elif high_freq_gain < low_freq_gain:
        if abs(high_freq_slope) < 30:
            return "Pasa bajas"
        else:
            return "Pasa bajas de orden superior"
    else:
        return "Pasa banda"

filter_type = determine_filter_type(mag_db, freq)

# Graficar respuesta en frecuencia completa
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

ax1.semilogx(freq, mag_db)
ax1.axvline(cutoff_freq, color='r', linestyle='--', alpha=0.5)
ax1.axhline(-3, color='g', linestyle='--', alpha=0.5)
ax1.set_title(f'Diagrama de Bode - Magnitud\nTipo de Filtro: {filter_type}')
ax1.set_xlabel('Frecuencia (Hz)')
ax1.set_ylabel('Magnitud (dB)')
ax1.grid(True)

ax2.semilogx(freq, phase)
ax2.set_title('Diagrama de Bode - Fase')
ax2.set_xlabel('Frecuencia (Hz)')
ax2.set_ylabel('Fase (grados)')
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"\nTipo de filtro identificado: {filter_type}")
print(f"Frecuencia de corte (-3dB): {cutoff_freq:.2f} Hz")

# Calcular el orden aproximado del filtro
slope_high_freq = np.mean(np.diff(mag_db[-100:]))/np.mean(np.diff(np.log10(freq[-100:])))
approx_order = abs(slope_high_freq/20)
print(f"Orden aproximado del filtro: {approx_order:.1f}")

# Características adicionales
print("\nCaracterísticas del filtro:")
print(f"Ganancia en bajas frecuencias: {mag_db[0]:.2f} dB")
print(f"Ganancia en altas frecuencias: {mag_db[-1]:.2f} dB")
print(f"Pendiente en la banda de transición: {slope_high_freq:.2f} dB/década")